In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Load the Excel file
file_path = "/content/Load Data in kW.xlsx"
df = pd.read_excel(file_path, header=None)

# Convert the dataframe into a NumPy array
load_data = df.values.flatten()

# Create input-output pairs (X: load from 24 hours ago, y: current load)
X = load_data[:-24].reshape(-1, 1)  # All values except the last 24
y = load_data[24:].reshape(-1, 1)   # All values except the first 24

# Normalize the data using Min-Max scaling
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)
y_scaled = scaler.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.1, random_state=42)

# Initialize parameters
m = 1.0  # Initial slope
c = -1.0  # Initial intercept
eta = 0.1  # Learning rate
gamma = 0.9  # Decay factor
epochs = 100

# Initialize moving averages of squared gradients
m_squared_grad = 0
c_squared_grad = 0
epsilon = 1e-8  # Small value to prevent division by zero

# Training loop
for epoch in range(epochs):
    # Compute predictions
    y_pred_train = m * X_train.flatten() + c

    # Compute gradients
    grad_m = -2 * np.mean((y_train.flatten() - y_pred_train) * X_train.flatten())
    grad_c = -2 * np.mean(y_train.flatten() - y_pred_train)

    # Update moving averages of squared gradients
    m_squared_grad = gamma * m_squared_grad + (1 - gamma) * grad_m**2
    c_squared_grad = gamma * c_squared_grad + (1 - gamma) * grad_c**2

    # Update parameters using RMSprop formula
    m -= (eta / (np.sqrt(m_squared_grad) + epsilon)) * grad_m
    c -= (eta / (np.sqrt(c_squared_grad) + epsilon)) * grad_c

    # Print progress every 10 epochs
    if (epoch + 1) % 10 == 0:
        loss = np.mean((y_train.flatten() - y_pred_train) ** 2)  # MSE loss

# Compute error metrics for training and test sets
y_train_pred = m * X_train.flatten() + c
y_test_pred = m * X_test.flatten() + c

mse_train = mean_squared_error(y_train, y_train_pred)
rmse_train = np.sqrt(mse_train)
mae_train = mean_absolute_error(y_train, y_train_pred)

mse_test = mean_squared_error(y_test, y_test_pred)
rmse_test = np.sqrt(mse_test)
mae_test = mean_absolute_error(y_test, y_test_pred)

# Display results
print("\nModel Performance Metrics:")
print(f"Training MSE: {mse_train:.6f}, RMSE: {rmse_train:.6f}, MAE: {mae_train:.6f}")
print(f"Test MSE: {mse_test:.6f}, RMSE: {rmse_test:.6f}, MAE: {mae_test:.6f}")



Model Performance Metrics:
Training MSE: 0.001477, RMSE: 0.038437, MAE: 0.032105
Test MSE: 0.019287, RMSE: 0.138879, MAE: 0.040743
